In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from skimage.util import img_as_ubyte
from skimage.io import imread
from skimage.morphology import square
from skimage.filters.rank import majority
from skimage.segmentation import quickshift
from rasterio.mask import mask
import rasterio as rio
from rasterio.merge import merge
from scipy.stats import mode

from glob import glob

from osgeo import gdal, ogr, osr
gdal.PushErrorHandler('CPLQuietErrorHandler')
gdal.UseExceptions()

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']

In [3]:
grid = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/Grid_Ekoreg_2022.gpkg")[['ID_GRID','geometry','kdPIC']]

In [4]:
from pysal.lib import weights

In [5]:
wr = weights.contiguity.Rook.from_dataframe(grid,idVariable='ID_GRID')

/opt/tljh/user/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 63 disconnected components.
 There are 10 islands with ids: ID-0702, ID-1404, ID-1829, ID-1836, ID-1890, ID-3719, ID-3941, ID-4731, ID-5159, ID-5525.
  warnings.warn(message)


In [6]:
grid['neighbor']=grid.ID_GRID.apply(lambda y: wr.neighbors[y])

In [7]:
from google.cloud import storage
client = storage.Client()


ls_=list(client.list_blobs('bps-gcp-bucket', prefix='MLST2023/result'))
result_done=[str(i).split('/')[-1].split(',')[0] for i in ls_ if str(i).split('/')[-1].split(',')[0]!='']

In [8]:
grid['main_done']=grid.ID_GRID.apply(lambda y:1 if 'classified_'+y+'.tif' in result_done else 0)

In [9]:
grid['neigh_done']=grid.neighbor.apply(lambda y: 1 if 
                                       set(['classified_'+j+'.tif' for j in y]).issubset(set(result_done)) else 0)

In [10]:
ls_2=list(client.list_blobs('bps-gcp-bucket', prefix='MLST2023/post-processing/major_filtering_segmentasi'))
result_done_post=[str(i).split('/')[-1].split(',')[0].replace('post_t_','').replace('.tif','') for i in ls_2 if str(i).split('/')[-1].split(',')[0]!='']

In [11]:
grid

,ID_GRID,geometry,kdPIC,neighbor,main_done,neigh_done
0,ID-1452,"MULTIPOLYGON (((105.07084 -6.42769, 105.25050 ...",E,"[ID-1453, ID-1484]",0,0
1,ID-1453,"MULTIPOLYGON (((105.07084 -6.60971, 105.25050 ...",E,"[ID-1452, ID-1454, ID-1485]",0,0
2,ID-1454,"MULTIPOLYGON (((105.07084 -6.79179, 105.25050 ...",E,"[ID-1453, ID-1486]",0,0
3,ID-1484,"MULTIPOLYGON (((105.25050 -6.42769, 105.43016 ...",E,"[ID-1452, ID-1517, ID-1485]",0,0
4,ID-1485,"MULTIPOLYGON (((105.25050 -6.60971, 105.43016 ...",E,"[ID-1518, ID-1453, ID-1484, ID-1486]",0,0
...,...,...,...,...,...,...
7192,ID-7189,"MULTIPOLYGON (((118.90489 -5.88200, 119.08456 ...",K,[ID-4025],0,0
7193,ID-7194,"MULTIPOLYGON (((106.14882 1.54406, 106.32848 1...",D,"[ID-7196, ID-7197]",0,0
7194,ID-7195,"MULTIPOLYGON (((106.32848 1.72501, 106.50814 1...",D,"[ID-7196, ID-7197]",0,0
7195,ID-7196,"MULTIPOLYGON (((106.14882 1.72501, 106.32848 1...",D,"[ID-7195, ID-7194]",0,0


In [12]:
grid_for_run=grid.query('main_done==1 and neigh_done==1').query('ID_GRID not in @result_done_post')

In [13]:
grid_for_run=grid_for_run.to_crs(3857)

In [14]:
grid_for_run

,ID_GRID,geometry,kdPIC,neighbor,main_done,neigh_done


In [15]:
dict_class={
    100101:1,
    100102:2,
    100103:3,
    100104:4,
    100105:5,
    100106:6,
    100107:7,
    100108:8,
    100201:9,
    100202:10,
    100203:11,
    100204:12,
    100205:13,
    100206:14,
    100301:15,
    100302:16,
    0:16
}
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    #print(
    #    f"File {source_file_name} uploaded to {destination_blob_name}."
    #)
from google.cloud import storage

In [16]:
for index, row in grid_for_run.iterrows():
    storage_client = storage.Client()
    geom_boundary=row['geometry']
    bucket = storage_client.get_bucket('bps-gcp-bucket')
    source_blob_name='MLST2023/result/classified_'+row['ID_GRID']+'.tif'
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename('ml_output/09_post_classification/temp_download/classified_'+row['ID_GRID']+'.tif')
    source_blob_name='citra-sentinel2/'+row['kdPIC']+'/duatahun_'+row['ID_GRID']+'_QALPN1_PakKus_sentinel2_10m.tif'
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename('ml_output/09_post_classification/temp_sentinel2/'+'duatahun_'+row['ID_GRID']+'_QALPN1_PakKus_sentinel2_10m.tif')    
    for j in row['neighbor']:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket('bps-gcp-bucket')
        source_blob_name='MLST2023/result/classified_'+j+'.tif'
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename('ml_output/09_post_classification/temp_download/classified_'+j+'.tif')
        pic=grid.query('ID_GRID==@j')['kdPIC'].values[0]
        source_blob_name='citra-sentinel2/'+pic+'/duatahun_'+j+'_QALPN1_PakKus_sentinel2_10m.tif'
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename('ml_output/09_post_classification/temp_sentinel2/'+'duatahun_'+j+'_QALPN1_PakKus_sentinel2_10m.tif')    
    y=row
    ls_data=y.neighbor
    ls_data.append(y['ID_GRID'])
    ls_merged_grid=list(set(list(ls_data)))
    ls_merged_class=['ml_output/09_post_classification/temp_download/classified_'+j+'.tif' for j in ls_merged_grid]
    ls_merged_sentinel2=['ml_output/09_post_classification/temp_sentinel2/'+'duatahun_'+j+'_QALPN1_PakKus_sentinel2_10m.tif' for j in ls_merged_grid]
    raster_to_mosiac=[]
    for p in ls_merged_class:
        raster = rio.open(p)
        raster_to_mosiac.append(raster)
    mosaic, output = merge(raster_to_mosiac)
    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        }
    )
    with rio.open('ml_output/09_post_classification/temp_merge_class/merged_classified_'+y.ID_GRID+'.tif', 
                  "w", **output_meta) as m:
        m.write(mosaic)
    mj_fil1=square(3)
    mj_fil2=square(3)
    data=imread('ml_output/09_post_classification/temp_merge_class/merged_classified_'+y.ID_GRID+'.tif', plugin='tifffile')
    data=np.vectorize(dict_class.get)(data)
    major_data=majority(data.astype(np.uint8),mj_fil1)
    major_data=majority(major_data,mj_fil2)
    with rio.open('ml_output/09_post_classification/temp_merge_class/merged_classified_'+y.ID_GRID+'.tif') as src:
        out_meta = src.meta
        out_meta.update({'driver':'GTiff',
                    'height':src.shape[0],
                    'width':src.shape[1],
                    'transform':src.transform})
        with rio.open('ml_output/09_post_classification/temp_postclass_1/post_t_'+y.ID_GRID+'.tif','w',**out_meta) as dest:
            dest.write(major_data,1)
    with rio.open('ml_output/09_post_classification/temp_postclass_1/post_t_'+y.ID_GRID+'.tif') as src:
        out_image, transformed = mask(src, y.geometry, crop=True,all_touched=True)
    with rio.open('ml_output/09_post_classification/temp_postclass_1/post_t_'+y.ID_GRID+'.tif') as src:
        out_image, out_transform = rio.mask.mask(src, geom_boundary, crop=True)
        out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    with rio.open('ml_output/09_post_classification/temp_postclass_1/post_t_'+y.ID_GRID+'.tif', "w", **out_meta) as dest:
        dest.write(out_image)
    
    with rio.open('ml_output/09_post_classification/temp_sentinel2/'+'duatahun_'+y.ID_GRID+'_QALPN1_PakKus_sentinel2_10m.tif') as src:
        out_meta = src.meta
        out_meta.update({'driver':'GTiff',
                     'count':3,
                     'height':out_image.shape[1],
                    'width':out_image.shape[2],
                    'transform':src.transform})
        array_=src.read([8,4,3])
        with rio.open('ml_output/09_post_classification/temp_segment1/to_segment_'+y.ID_GRID+'.tif','w',**out_meta) as dest:
            dest.write(array_)
    data=imread('ml_output/09_post_classification/temp_segment1/to_segment_'+y.ID_GRID+'.tif', plugin='tifffile')
    segment_=quickshift(data,kernel_size=1,max_dist=5)
    data_=pd.DataFrame({'data':np.ravel(out_image[0]),'segment':np.ravel(segment_)})
    y_pivot=pd.pivot_table(data_,index='segment',values='data',aggfunc=mode)
    y_pivot['data_']=y_pivot.data.apply(lambda y: y[0][0])
    dict_done=y_pivot['data_'].to_dict()
    data_['post_label']=data_.segment.apply(lambda y:dict_done[y])
    data_done_=data_[['post_label']].to_numpy().reshape(out_image.shape)
    out_meta.update({'count':1})
    with rio.open('ml_output/09_post_classification/result_post/post_'+y.ID_GRID+'.tif','w',**out_meta) as dest:
        dest.write(data_done_)
    upload_major=glob('ml_output/09_post_classification/temp_postclass_1/*')
    for i in upload_major:
        y_=i.split('/')[-1]
        upload_blob('bps-gcp-bucket',i,'MLST2023/post-processing/major_filtering/'+y_)
    upload_major_segment=glob('ml_output/09_post_classification/result_post/*')
    for i in upload_major:
        y_=i.split('/')[-1]
        upload_blob('bps-gcp-bucket',i,'MLST2023/post-processing/major_filtering_segmentasi/'+y_)